# As usual start with the imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import cos, arccos, sin, pi, round
from numpy.linalg import matrix_rank as rank
from numpy.linalg import svd
from scipy.linalg import orth
!rm bug_numpy_utils.py # just in case
!wget https://raw.githubusercontent.com/bugrakoku/bug_python_utils/main/bug_numpy_utils.py
from bug_numpy_utils import MatPrint, CData # note that once these files are downloaded you can read their content.
!wget https://raw.githubusercontent.com/bugrakoku/bug_python_utils/main/me536utils.py
from me536utils import RotMat

## Generate data in 2D, so that it is easily plottable
Data from a sort of elliptical regions is generated.
It is recommended that you generate other forms of data and play with the following cells.
Let $\mathbf{D}$ be the data matrix.

In [ ]:
# generate data on an elliptic region in 2D
N = 100# number of data points
#'''
# random data
AR = 5 # aspect ratio of the ellipse
D = np.random.randn(2,N)
B = orth(np.random.rand(2,2))
B[:, 0] *= AR # let the first random axis be the longer one
D = B @ D
title = 'kinda elliptic, or linear or so...'
#'''

'''
# generate circle
r = 5
n = 0
D = np.array([[r*cos(theta), r*sin(theta)] for theta in np.linspace(-pi,pi,N)] ).T
title = 'circle'
#'''
CData(D, title)

## $\mathbf{D}$ is the original data matrix
Now randomly translate it to a $d$ dimensional space to obtain $\mathbf{D}_d$, and check the rank and singular values of $\mathbf{D}_d$.  
Observe the number of non-zero singular values and explain why?  

Also run the below cell several times and check out the singular values. Even though the basis $\mathbf{B}_b$ is random, why are the singular values the way they are?

In [ ]:
# get a random basis for d-dimensional space
d = 10 # you can play with d, but in general the idea is that it should be larger than 3
Bd = orth(np.random.rand(d,2)) # basis to project up to higher dimensions, let's save it just in case, but why is it (d,2)?
Dd = Bd @ D # project 2D data to d-D
print(f'Columns of Dd come from {d}-dimensional space yet columns are trapped in a {rank(Dd)}-dimensional subspace')
U,S,Vt = svd(Dd) #'lets check out the singular values
print(f'\nsingular values: {np.round(S,3)}')

## let's transfer $\mathbf{D}_d$ away from the origin
Add a random displacement vector to $\mathbf{D}_d$ to obtain $\mathbf{D}_{da}$ and check the rank and singular values again. Make sure that the new rank and singular values meet your expectations.  

Also run the following cell many times, and check out the singular values. What is different?

While testing also change the ```distMultiplier``` to values much higher and lower and observe singular value change.

In [ ]:
distMultiplier = 10
dispV =  np.random.rand(d,1) * distMultiplier
Dda = Dd + dispV # subspace Dd becomes affine
print(f'Columns of Dd come from {d}-dimensional space yet columns are trapped in a {rank(Dda)}-dimensional subspace, or are they?\n')
U,S,Vt = svd(Dda)
print(f'singular values: {np.round(S,3)}')

## Finally add some noise to $\mathbf{D}_{da}$ to obtain $\mathbf{D}_{dan}$
add a factor to scale noise level  
observe how the lower 7 singular values are no more 0!  

We are done with generation of data. Next, we will start moving them back to $2D$

In [ ]:
nl = 0.01 # change noise level via this
Ddan = Dda + np.random.randn(*Dda.shape) * nl
print(f'Columns of Dd come from {d}-dimensional space yet columns are trapped in a {rank(Ddan)}-dimensional subspace, or are they?')
U,S,Vt = svd(Ddan)
MatPrint(np.round(S,3), '\nsingular values of the noisy matrix')


# This following is the reconstruction of the 2D data from $d-dimensional$ space

Start using the non-noisy, subspace data $\mathbf{D}_d$ first, step by step get the data back to 2D and compare your reconstructed matrix in $2D$ with the original $\mathbf{D}$.

Then come back and this time use the data in the affine space, i.e. $\mathbf{D}_{Da}$, and finally use the one that is noisy, i.e. $\mathbf{D}_{Dan}$.  

Note that you can change the noise level above to see what happens in your data rescue mission if noise levels get high.

### Start with the non-noisy case: $\mathbf{D}_d$
and work your way back to 2D to get the best possible match

In [ ]:
# use either the noisy or the clean matrix
DT = Dd
#DT = Dda # use data from affine space next
#DT = Ddan # use noisy data last

# remember zero mean before anything, evenif you think it is not needed
zm = np.mean(DT, axis=1).reshape((d,1)) # get the mean

MatPrint(zm-dispV, 'zero mean diff') # WHY IS THIS NOT ZERO? evaluate this for different cased i.e. Dd, Dda, Ddan

Dd2 = DT - zm

U, S, Vt = svd(Dd2)

# now write everything with respect to the U using its first 2 columns
C = np.diag(S)[0:2,0:2] @ Vt[0:2,:]

## Check out the combined rank
Before even running the following block, read the code, and guess how the singular values are going to come out.

In [ ]:
# Check out that U and Bd are spanning the same plane in case of no noise
# almost the same plane in case of noise
# first print the 2 different basis vectors
MatPrint(U[:,0:2], 'First 2 columns of U')
MatPrint(Bd, '\nOriginal basis')

# now check out the singular values of the combined matrix
U, S, Vt = svd(np.hstack((U[:,0:2], Bd)))
MatPrint(S, '\nSingular values of the combined matrix')

### Plot the original and the recovered

In [ ]:
plt.plot(D[0,:], D[1,:], 'r*')
plt.plot(C[0,:], C[1,:], 'g+')
plt.legend(['original', 'recovered'])

## Why is the recovered data aligned with the $X-axis$?
Think about how you can align the recovered data to the best you can with the original?

Hint: Check out the first left singular values?

In [ ]:
# align both matrices to the best you can

##Note that:
Even when there is no noise, when data matrices are alined using first singular values as reference from both matrices, they might not perfectly align!
When would this occur and how can you solve this problem?

In [ ]:
# this time align for sure.